# Taller de repaso: Aprendizaje no supervisado - parte 2

## Preparación del notebook

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score
#!pip install kneed
from kneed import KneeLocator

from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA, FactorAnalysis

#!pip install factor_analyzer
from factor_analyzer import FactorAnalyzer

#!pip install plotly
import plotly.express as px

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Lectura del dataset

In [3]:
# Lectura del dataset
df = pd.read_csv('https://raw.githubusercontent.com/mlondono-oc/LEA2/main/Modulo-3/data/train.csv')
df.head(3)

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied


## Pre-procesamiento

In [ ]:
# Eliminación de variables no significativas
df.drop(['Unnamed: 0', 'id', 'Arrival Delay in Minutes'], axis=1, inplace=True)
# Tomas una muestra aleatoria
df_sample = df.sample(n=2500, random_state=123)
df_sample.reset_index(drop=True, inplace=True)

# Codificación de variable satisfacción
train_df1 = df_sample.copy()
train_df1['satisfaction'] = train_df1['satisfaction'].map({'neutral or dissatisfied':0, 'satisfied':1})
train_df1['Gender'] = train_df1['Gender'].map({'Female':0, 'Male':1})
train_df1['Customer Type'] = train_df1['Customer Type'].map({'disloyal Customer':0, 'Loyal Customer':1})
train_df1['Type of Travel'] = train_df1['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
train_df1['Class'] = train_df1['Class'].map({'Eco':1, 'Eco Plus':2, 'Business':3})

# Escalado de variables numéricas
train_cols = train_df1[['Age', 'Flight Distance', 'Departure Delay in Minutes']]
scaler = StandardScaler()
scaled_train_df1 = pd.DataFrame(scaler.fit_transform(train_cols), columns = train_cols.columns)

#  Union de data numéricas  y categórica
cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes']
train_df1[cols] = scaled_train_df1

# Tratamiento de valores atípicos
train_df1_out = train_df1.copy()
# Identificación de valores atípicos
media = train_df1_out.mean()
std = train_df1_out.std()
lim_superior = media + 3 * std
lim_inferior = media - 3 * std
outliers = train_df1_out[(train_df1_out > lim_superior)|(train_df1_out < lim_inferior)].stack()
train_df1_out = train_df1_out.drop(outliers.index.get_level_values(0))
train_df1_out.head(3)

## Reducción de la dimensionalidad

In [ ]:
#Cree un dataset reducido que proporcione el 85% de la varianza de entrada que debe explicarse
pca = PCA(n_components=3).fit(train_df1_out)

#Por último, transforme "train_df1"
X_pca = pca.fit_transform(train_df1_out)
X_pca[:3]

In [ ]:
df_final = train_df1_out.copy()
df_final['PCA1'] = X_pca[:,0]
df_final['PCA2'] = X_pca[:,1]
df_final['PCA3'] = X_pca[:,2]
df_final.head(3)

Las loadings o cargas pueden interpretarse como el peso/importancia que tiene cada variable en cada componente y, por lo tanto, ayudan a conocer que tipo de información recoge cada una de las componentes.

In [ ]:
# --- Pesos de las variables que componen las componentes principales ---
pesos_pca = pd.DataFrame(pca.components_, columns = train_df1_out.columns,
             index = ['PC 1', 'PC 2', 'PC 3']).round(2).T

pesos_pca

## Modelo K-means

In [ ]:
# Número optimo de K
inertia_list = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=3)
    kmeans.fit(X_pca)
    inertia_list.append(kmeans.inertia_)

# plot the inertia curve
plt.plot(range(1,11),inertia_list)
plt.scatter(range(1,11),inertia_list)
plt.xlabel("Number of Clusters", size=10)
plt.ylabel("Inertia Value", size=10)
plt.title("Different Inertia Values for Different Number of Clusters", size=12)
plt.show()

In [ ]:
kmeans_constants = {"init": "k-means++", "n_init": 100, "max_iter": 500, "random_state": 42}

# --- Modelo K-means ---
model_kmeans = KMeans(n_clusters = 4, **kmeans_constants)
model_kmeans.fit(X_pca)

In [ ]:
# --- Evaluación del modelo kmeans ---
print(" ### K-MEANS ###")
print('Inertia: ', model_kmeans.inertia_)
print('Silhouette Score: ', silhouette_score(X_pca, model_kmeans.labels_))
print('Calinski harabasz score: ', calinski_harabasz_score(X_pca, model_kmeans.labels_))

In [ ]:
# Predicción de los clusters
Cluster_1 = model_kmeans.fit_predict(X_pca)

df_final['Cluster_1']= Cluster_1

# A partir de ahora, sólo trabajaremos con "df_final"
df_final.head(3)

In [ ]:
# 3d scatterplot using plotly.express
fig = px.scatter_3d(df_final, x="PCA1", y="PCA2", z="PCA3", color="Cluster_1")
fig.update_layout(width=1000, height=700)
fig.show()

In [ ]:
fig = px.scatter(df_final, x="PCA1", y="PCA2", color="Cluster_1")
fig.update_layout(width=1000, height=700)
fig.show(config={'displayModeBar': False})

## Interpretación clusters

In [ ]:
# Conversión de variables numéricas a valores originales
train_cols = train_df1_out[['Age', 'Flight Distance', 'Departure Delay in Minutes']]
scaled_train_df1 = pd.DataFrame(scaler.inverse_transform(train_cols), columns = train_cols.columns)

# Union de data numéricas y categórica
cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes']
df_final[cols] = scaled_train_df1
df_final.head(3)

In [ ]:
# Composición de los clusters formados
sns.countplot(x="Cluster_1",data=df_final, palette="Set2", order = df_final['Cluster_1'].value_counts().index)

In [ ]:
# Composición de los clusters - Media de las variables por cluster
df_final.groupby(['Cluster_1']).mean()

In [ ]:
# Variables Categóricas
cat_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction']

pal=['autumn','cool','magma','spring','winter'] # 5 color palettes for 5 categorical vars
    
fig,axs=plt.subplots(ncols=5,figsize =(30, 10))

for idx,i in enumerate(df_final[cat_cols].columns):
    plt.title(i)
    sns.histplot(data=df_final,x='Cluster_1', hue=i, palette=pal[idx], binwidth=.5, ax=axs[idx] ,multiple='dodge')

In [ ]:
# Análisis de la edad
temp_edad = df_final[['Cluster_1','Age']]
fig,axs=plt.subplots(ncols=4,figsize =(25, 10))

for i in list(range(0,4)):
    plt.title(i)
    sns.histplot(data=temp_edad[temp_edad.Cluster_1==i],x='Age',hue='Cluster_1',palette=pal[i],binwidth=.8,ax=axs[i],multiple='dodge')

In [ ]:
# Análisis de la distancia recorrida
temp_distancia = df_final[['Cluster_1','Flight Distance']]
fig,axs=plt.subplots(ncols=4,figsize =(25, 10))

for i in list(range(0,4)):
    plt.title(i)
    sns.histplot(data=temp_distancia[temp_distancia.Cluster_1==i],x='Flight Distance',hue='Cluster_1',palette=pal[i],binwidth=.8,ax=axs[i],multiple='dodge')

In [ ]:
# Analisis de la encuesta
temp_encuesta = df_final.columns[6:20]
df_final["Average_point"]=round(df_final[temp_encuesta].mean(axis=1),3)

fig,axs=plt.subplots(ncols=4,figsize =(25, 10),sharex=True)

for i in list(range(0,4)):
    plt.title(i)
    sns.histplot(data=df_final[df_final.Cluster_1==i],x="Average_point",hue="Cluster_1",palette=pal[i],binwidth=.5,ax=axs[i],multiple='dodge')

In [ ]:
df_final[df_final['Cluster_1']==3]['Customer Type'].value_counts(normalize=True)

###  Summary

**Cluster 0**


**Cluster 1**


**Cluster 2**


**Cluster 3**



## Análisis de  factores

### Prueba de adecuación

La prueba de esfericidad de Bartlett verifica si las variables observadas se intercorrelacionan o no utilizando la matriz de correlación observada contra la matriz de identidad. Si la prueba resulta estadísticamente insignificante, no se debe emplear un análisis factorial.

En esta prueba de Bartlett, el valor p es 0. La prueba fue estadísticamente significativa, lo que indica que la matriz de correlación observada no es una matriz de identidad.

In [ ]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

# Selección de variables de encuestas
df_fa = train_df1_out[train_df1_out.columns[6:20]] 

# Aplicación de la prueba bartlett_sphericity
chi_square_value, p_value = calculate_bartlett_sphericity(df_fa)
chi_square_value, p_value

La prueba KMO mide la idoneidad de los datos para el análisis factorial. Determina la adecuación para cada variable observada y para el modelo completo. KMO estima la proporción de varianza entre todas las variables observadas. Una proporción más baja es más adecuada para el análisis factorial. Los valores de KMO oscilan entre 0 y 1. Un valor de KMO inferior a 0,6 se considera inadecuado.

El KMO general de nuestros datos es 0.78, lo cual es excelente. Este valor indica que puede continuar con el análisis factorial planificado.

In [ ]:
from factor_analyzer.factor_analyzer import calculate_kmo

# Aplicación de la prueba KMO
kmo_all,kmo_model=calculate_kmo(df_fa)
kmo_model

### Elección del número de factores

In [ ]:
# Crear un objeto de análisis factorial y realizar un análisis factorial
fa = FactorAnalyzer(15, rotation=None)
fa.fit(df_fa)

# Check Eigenvalues
ev, v = fa.get_eigenvalues()
ev

In [ ]:
# Diagrama con valores propios
plt.scatter(range(1,df_fa.shape[1]+1),ev)
plt.plot(range(1,df_fa.shape[1]+1),ev)
plt.title('Scree Plot')
plt.xlabel('Factors')
plt.ylabel('Eigenvalue')
plt.grid()
plt.show()

Aquí sólo utilizaremos 3 factores, dada la gran caída del valor propio después del tercer factor. Veamos qué factores se crean y qué variables contienen.

### Aplicación de análisis factorial

In [ ]:
# Analisis factorial
fa = FactorAnalyzer(3, rotation='varimax')
fa.fit(df_fa)

# Cargas
loads = fa.loadings_

# Cargas por cada variable y factor
print(pd.DataFrame(fa.loadings_,index=df_fa.columns))

He aquí los 3 factores, las variables que contienen y su posible "interpretabilidad":

* Comodidad: Comida y bebida, Comodidad del asiento, Entretenimiento a bordo, Limpieza

* Servicio: Servicio a bordo, Gestión de equipajes, Servicio a bordo

* Conveniencia: Wifi en vuelo, Comodidad de la hora de salida/llegada, Reserva en línea, Ubicación de la puerta de embarque.

In [ ]:
# Varianza explicada por cada factor
print(pd.DataFrame(fa.get_factor_variance(),index=['Variance','Proportional Var','Cumulative Var']))

In [ ]:
# Análisis de comunalidad
print(pd.DataFrame(fa.get_communalities(),index=df_fa.columns,columns=['Communalities']))

In [ ]:
# Transformación del dataset
X_fa = fa.fit_transform(df_fa)
X_fa.shape